##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Convolutional Neural Network (CNN)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/cnn">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/cnn.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/cnn.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates training a simple [Convolutional Neural Network](https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) (CNN) to classify [CIFAR images](https://www.cs.toronto.edu/~kriz/cifar.html). Because this tutorial uses the [Keras Sequential API](https://www.tensorflow.org/guide/keras/overview), creating and training your model will take just a few lines of code.


### Import TensorFlow

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

import os

import numpy as np
from PIL import Image
import fnmatch

import matplotlib.pyplot as plt

np.random.seed(42)


### Download and prepare the CIFAR10 dataset


The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

In [ ]:
#

from Distortion_Functions import load_distortion_data

image_width = 240
image_height = 320

# directory
dir_path = 'Training Set GS 1000 320x240'


# dist_images, dist_labels, dist_values, dist_filenames = load_distortion_data(dir_path, image_width, image_height)

dist_data = load_distortion_data(dir_path, image_width, image_height)

dist_images = dist_data["dist_images"]
dist_labels = dist_data["dist_labels"]
dist_values = dist_data["dist_values"]
dist_filenames = dist_data["dist_filenames"]


In [ ]:
# Remove specified amount of undistorted images

from Distortion_Functions import remove_data_by_label

dist_data = remove_data_by_label(dist_data, 0, 401)



In [ ]:
#

from Distortion_Functions import bin_dist_values

num_bins = 5

dist_data = bin_dist_values(dist_data, num_bins, None, None, False)

dist_value_bins = dist_data["dist_value_bins"]

# print(dist_values[:,0])

# print(dist_value_bins[:,0])

# dist_value_bins

In [ ]:
#

from Distortion_Functions import dist_train_test_split

split_data = dist_train_test_split(dist_data, 0.8, 42)

idx_train = split_data["idx_train"]
train_images = split_data["train_images"]
train_labels = split_data["train_labels"]
train_values = split_data["train_values"]
train_filenames = split_data["train_filenames"]
train_bins = split_data["train_bins"]

idx_test = split_data["idx_test"]
test_images = split_data["test_images"]
test_labels = split_data["test_labels"]
test_values = split_data["test_values"]
test_filenames = split_data["test_filenames"]
test_bins = split_data["test_bins"]

split_data.keys()


### Verify the data

To verify that the dataset looks correct, let's plot the first 25 images from the training set and display the class name below each image:


In [ ]:
#

class_names = ['undistorted', 'distorted']

plt.figure(figsize=(18,18))
for k1 in range(0,24):
    plt.subplot(6,4,k1+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[k1])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    # plt.xlabel(f"{filenames[idx_train[k1]].split('.')[0]} : {class_names[train_labels[k1,0]]}")
    # plt.xlabel(f"{idx_train[k1]} | {dist_filenames[idx_train[k1]].split('.')[0]} | {dist_values[idx_train[k1],0]} | {dist_value_bins[idx_train[k1],0]}")
    plt.xlabel(f"{k1} | {train_filenames[k1].split('.')[0]} | {train_values[k1,0]} | {train_bins[k1,0]}")
plt.show()


### Create the convolutional base

The 6 lines of code below define the convolutional base using a common pattern: a stack of [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) and [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure your CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument `input_shape` to your first layer.


In [ ]:
# Create first model

model_01 = models.Sequential()
model_01.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(dist_images.shape[1], dist_images.shape[2], 1)))
model_01.add(layers.MaxPooling2D((2, 2)))
model_01.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_01.add(layers.MaxPooling2D((2, 2)))
model_01.add(layers.Conv2D(64, (3, 3), activation='relu'))

model_01.add(layers.Flatten())
model_01.add(layers.Dense(64, activation='relu'))
model_01.add(layers.Dense(num_bins, activation='softmax'))
# model_01.add(layers.Dense(1, activation='sigmoid'))

model_01.summary()


In [ ]:
# Compile and train first model

# compile model
model_01.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

# train model
history = model_01.fit(train_images, train_bins, epochs=12, validation_data=(test_images, test_bins))


### Evaluate the model

In [ ]:
# dir(history.model.history)

# history.model.history

In [ ]:
#

fig1 = plt.figure(figsize=(12,6))
# plt.plot(history.epoch,0.6*np.ones(len(history.epoch)),'k--')
# plt.plot(history.epoch,0.7*np.ones(len(history.epoch)),'k--')
# plt.plot(history.epoch,0.8*np.ones(len(history.epoch)),'k--')
# plt.plot(history.epoch,0.9*np.ones(len(history.epoch)),'k--')
# plt.plot(history.epoch,1.0*np.ones(len(history.epoch)),'k--')
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label = 'test accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.2, 1.2])
plt.grid()
plt.legend(loc='lower right')

test_loss, test_acc = model_01.evaluate(test_images, test_bins, verbose=2)


In [ ]:
# Look at output of model for single image

kim = 24

prediction = model_01.predict(test_images[kim].reshape(1,image_width,image_height))[0]
print(f"Model prediction for image {kim}:")
print(prediction)

# apply classification using argmax to 0 or 1
predicted_bins = np.argmax(prediction)

# print(f"Predicted bins for test image: {predicted_bins} | {class_names[predicted_bins]}")
print(f"Predicted bins for test image: {predicted_bins}")
# print(predicted_bins)

# print(f"True class for terst image: {test_labels[kim,0]} | {class_names[test_labels[kim,0]]}")
print(f"True class for test image: {test_bins[kim,0]}")

print(f"Distortion value for test image: {test_values[kim,0]}")
# print(test_labels[kim,0].T)

fig1 = plt.figure(figsize=(8,8))
plt.imshow(test_images[kim])
plt.xlabel(f"{test_filenames[kim]} - Distortion Coefficient: {test_values[kim,0]} | Predicted label: {predicted_bins} | Actual label: {test_bins[kim,0]} | ")
plt.show()


In [ ]:
# Validate model accuracy

# model prediction from test_images
prediction = model_01.predict(test_images)
# print(prediction)

# apply classification using argmax to 0 or 1
predicted_bins = np.argmax(prediction, axis=1)

print("Predicted bins for test images:")
print(predicted_bins)

print("Test image bins:")
print(test_bins[:,0].T)

print(f"Number of correct predictions out of number of test images: {np.sum(predicted_bins == test_bins[:,0])} / {len(test_bins)}")
# np.sum(predicted_bins == test_labels)

print(f"Model accuracy on test images: {np.sum(predicted_bins == test_bins[:,0]) / len(test_bins)}")

# wrong_idx = np.asarray(predicted_bins != test_labels[:,0], dtype=int)
# wrong_idx = np.arange(0,len(wrong_idx))[wrong_idx == 1]

wrong_idx = np.arange(0,len(test_images))[(predicted_bins != test_bins[:,0])]
print("Indices for erroneous predictions")
print(wrong_idx)
# print("Filenames for errorneous predictions")
# print(np.array(test_filenames)[wrong_idx])

print("Distortion coefficients for erroneous predictions:")
print(test_values[wrong_idx,0])

print("Distortion bins for erroneous predictions:")
print(test_bins[wrong_idx,0])

print("Predicted bins for erroneous predictions:")
print(predicted_bins[wrong_idx])

# print(f"True negatives: {np.sum((predicted_bins == 0))} / {np.sum((test_labels[:,0] == 0))}")
# print(f"True positives: {np.sum((predicted_bins == 1))} / {np.sum((test_labels[:,0] == 1))}")

# print(f"True negatives: {np.sum((predicted_bins[und_idx] == 0))} / {np.sum((test_bins[:,0] == 0))}")
# print(f"True positives: {np.sum((predicted_bins[dis_idx] == 1))} / {np.sum((test_bins[:,0] == 1))}")
# print(f"False negatives: {np.sum((predicted_bins[dis_idx] == 0))} / {np.sum((test_bins[:,0] == 1))}")
# print(f"False positives: {np.sum((predicted_bins[und_idx] == 1))} / {np.sum((test_bins[:,0] == 0))}")

fig1 = plt.figure(figsize=(12,4))
counts, bins = np.histogram(test_bins[:,0], num_bins)#, (-0.5,3.5))
plt.stairs(counts, bins)
ax = plt.gca()
for k1 in range(0,len(bins)-1):
    ax.text(x=(bins[k1]+bins[k1+1])/2, y=counts[k1], s=counts[k1], va='center', ha='center', size='xx-large')
plt.xlabel("Bin")
plt.ylabel("Number in bin")
plt.show()

fig1 = plt.figure(figsize=(12,4))
# counts, bins = np.histogram(dist_value_bins[:,0], num_bins, (0, num_bins))
# print(bins)
# print(counts)
# plt.stairs(counts, bins)
for k1 in range(0,np.max(test_bins[:,0])+1):
    plt.plot(k1, np.sum(test_bins[:,0] == k1), "b.")
plt.xlabel("Bin")
plt.ylabel("Number in bin")
plt.show()

conf_matrix = np.zeros((num_bins,num_bins))
for k1 in range(0,num_bins):
    for k2 in range(0,num_bins):
        temp_idx = (test_bins[:,0] == k1)
        conf_matrix[k1,k2] = np.sum((predicted_bins[temp_idx] == k2))
        
print("Confusion Matrix:")
print(conf_matrix)

# plot the confusion matrix using Matplotlib
fig1, ax = plt.subplots(figsize=(8, 8))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for k1 in range(conf_matrix.shape[0]):
    for k2 in range(conf_matrix.shape[1]):
        ax.text(x=k2, y=k1, s=conf_matrix[k1, k2], va='center', ha='center', size='xx-large')
plt.xlabel('Predicted Values', fontsize=18)
plt.ylabel('Actual Values', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()


In [ ]:
#####


In [ ]:
# Create second model

model_02 = models.Sequential()
model_02.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(dist_images.shape[1], dist_images.shape[2], 1)))
model_02.add(layers.MaxPooling2D((2, 2)))
model_02.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_02.add(layers.MaxPooling2D((2, 2)))
model_02.add(layers.Conv2D(64, (3, 3), activation='relu'))

model_02.add(layers.Flatten())
model_02.add(layers.Dense(64, activation='relu'))
model_02.add(layers.Dense(16, activation='relu'))
model_02.add(layers.Dense(4, activation='relu'))
model_02.add(layers.Dense(1, activation='relu'))

model_02.summary()


In [ ]:
# Compile and train first model

# compile model
model_02.compile(optimizer='sgd', loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])

# train model
history = model_02.fit(train_images, train_values, epochs=128, validation_data=(test_images, test_values))
